---
title: "Simulation &<br>Agent-based Models"
subtitle: "[DAY FIVE]{.kn-pink} [GESIS Fall Seminar in Computational Social Science]{.kn-blue}"
author:
  - name: John McLevey
    affiliations:
      - name: University of Waterloo
  - name: Johannes B. Gruber
    affiliations:
      - name: VU Amsterdam
output-dir: "../docs/"
format:
  revealjs:
    theme: [default, custom.scss]
    width: 1600
    height: 900
    embed-resources: true
    execute:
      echo: true
      warning: false
      cache: true
      freeze: true
    slide-number: false
    chalkboard: false
    preview-links: auto
    smaller: true
    fig-align: left
    fig-format: svg
    lightbox: true
    scrollable: true
    code-overflow: scroll
    code-fold: false
    code-line-numbers: true
    code-copy: hover
    reference-location: document
    tbl-cap-location: margin
    logo: media/logo_gesis.png
    footer: "[CC BY-SA 4.0]{.nord-footer}"
    email-obfuscation: javascript
highlight-style: "nord"
bibliography: references.bib
---


<!-- {{< include _day-5-introduction.qmd >}} -->
<!-- {{< include _day-5-synthetic-networks.qmd >}} -->

## Agent-based Modelling in Python

`NDLib`

<br>

:::: {.columns}
::: {.column width="40%"}

![Don't forget about your conda environment!](media/chatgpt_dependency_hell.png){width=100%}

:::

::: {.column width="5%"}
:::

::: {.column width="55%"}
With the `gt` conda environment still activated, import the packages.


In [ ]:
import networkx as nx
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import ndlib.models.ModelConfig as mc 
import ndlib.models.epidemics as ep 
from ndlib.utils import multi_runs

from icsspy import set_style
import icsspy.simulations as sims

set_style()

:::
::::


##


In [ ]:
population_size = 300 
G = nx.watts_strogatz_graph(population_size, 4, 0.15) 
G.name = "A Simulated Small World" 

layout = nx.nx_pydot.graphviz_layout(G) 

fig, ax = plt.subplots(figsize=(12, 12)) 
nx.draw(G, pos=layout, node_color='gray', edge_color='gray', node_size=100, width=.5) 
plt.show()

##

We'll define a function


In [ ]:
def sir_model(network, beta, gamma, fraction_infected): 
  config = mc.Configuration() 
  config.add_model_parameter('beta', beta) 
  config.add_model_parameter('gamma', gamma) 
  config.add_model_parameter("fraction_infected", fraction_infected) 
  
  model = ep.SIRModel(network) 
  model.set_initial_status(config) 
  return model

## 

And run our first SIR model!


In [ ]:
sir_model_1 = sir_model(G, beta=0.05, gamma=0.01, fraction_infected=0.1) 
sir_1_iterations = sir_model_1.iteration_bunch(200, node_status=True)
sir_1_trends, sir_1_deltas = sims.simulation_df(sir_1_iterations, G)
sir_1_trends.head(10)

##


In [ ]:
fig, ax = plt.subplots()
sns.lineplot(data=sir_1_trends)
ax.set(xlabel='Iteration / step in time', ylabel='Proportion of nodes')
sns.despine()
plt.legend()
plt.show()

##


In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3,1, figsize=(8,8), sharex=True)
sns.lineplot(ax = ax1, data=sir_1_deltas['Susceptible'], color='gray')
sns.lineplot(ax = ax2, data=sir_1_deltas['Infected'], color='gray')
sns.lineplot(ax = ax3, data=sir_1_deltas['Removed'], color='gray')
## EMPHASIZE DEVIATIONS FROM 0
ax1.axhline(0, color='crimson')
ax2.axhline(0, color='crimson')
ax3.axhline(0, color='crimson')
ax3.set(xlabel='Iteration')
plt.show()

##

![The same simulated small-world network as before, displayed 20 times –
once for each of the first 20 time steps in the simulation of epidemic spread through the
network](media/small_world_epidemic.png){width=100% #fig-small_world_epidemic}

## Let's do that 500 more times


In [ ]:
trends = multi_runs(
  sir_model_1, 
  execution_number=3, 
  iteration_number=200, 
  infection_sets=None, 
  nprocesses=4
)

sims.visualize_trends(trends, network=G, proportion=True, return_data=False)

# References

## 